In [175]:
import numpy as np 
import pandas as pd 
import sys
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import StackingRegressor

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

import time
import os

from pandas import Series
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm
from datetime import timedelta, datetime, date

import json

In [176]:
# фиксируем RANDOM_SEED
RANDOM_SEED = 42

In [177]:
# Подготовим функции для простоты дальнейшей работы:

def visualizing_number(data, column): 
    bins = 100
    if data[column].nunique() < 100:
        bins = data[column].nunique()
    
    fig, axes = plt.subplots(2,1, figsize=(26,8))
    sns.boxplot(ax=axes[0], y = data[column],data=data, orient='h')
    axes[0].set_title(column)
    data[column].hist(ax=axes[1], bins = bins)
    plt.show()
    print()
    

def get_outliers(column): 
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    min_out = Q1 - 1.5 * IQR
    max_out = Q3 + 1.5 * IQR
    return (column < min_out).sum() + (column > max_out).sum(), min_out, max_out

def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [178]:
DIR_TRAIN = '../input/module6/' #импортируем данные, которые были предвариельно взяты с сайта авто.ру
DIR_TEST   = '../input/sf-dst-car-price-prediction/'

In [179]:
train_2020 = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv')
train = pd.read_csv(DIR_TRAIN+'cars_df.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [180]:
train_2020.info()

In [181]:
train.info()

In [182]:
test.info()

#### В файле 2020 года объявлений гораздо больше, чем в датасете 2021 года. Однако спарсить большее количество не удалось. Также цены в 2021 году сильно изменились. Учтем инфляцию и объединим датасеты 2020 и 2021 года.

In [183]:
# удалим знак валюты и пробелы в цене

train['price'] =train['price'].astype(str)
train['price'] = train['price'].apply(lambda x: x if len(x) ==1 else ''.join(x.split()[0:-1])).astype(int)

In [184]:
# инфляция по брендам

brands = test.brand.unique()
infs = []
for brand in brands:
    mean20 = train_2020[train_2020["brand"]==brand].price.mean()
    mean21 = train[train["brand"]==brand].price.mean()
    inf = (mean21-mean20)/mean20
    infs.append(inf)
    print(f'{brand} 2020: {mean20}')
    print(f'{brand} 2021: {mean21}')
    print(f'Инфляция: {round(inf * 100,2)}%')
    print()

In [185]:
# Учтем инфляцию и изменим цены так, чтобы они были похожи на осень 2020 года

brands = test.brand.unique()
tt = train.copy()
display(tt.head(2))
for brand in brands:
    mean20 = train_2020[train_2020["brand"]==brand].price.mean()
    mean21 = train[train["brand"]==brand].price.mean()
    inf = (mean21-mean20)/mean20
    if inf>0:
        tt.loc[tt["brand"]==brand, ['price']] = tt.loc[tt["brand"]==brand, ['price']].apply(lambda x: round(x/(1+inf),0))
display(tt.head(2))

In [186]:
# Цены изменились верно
data = tt

In [187]:
print(train.columns)
print(train_2020.columns)
print(test.columns)

In [188]:
# Создадим список общих переменных для теста и трейна:

s2020 = list(train_2020.columns)
s2021 = list(train.columns)
s = set(list(test.columns))

no2020 = [x for x in s if not x in s2020] # найдем стобцы, которых нет в трейне
print('Столбцы, которые есть в тесте, но их нет в трейне 2020:', no2020)

no2021 = [x for x in s if not x in s2021] # найдем стобцы, которых нет в трейне
print('Столбцы, которые есть в тесте, но их нет в трейне 2021:', no2021)

# посмотрим, какие признаки отличают два датасета

# model_info дублирует полезную информацию из столбца car_url, 
# super_gen - аккумулирует и дублирует много информации с разных стобцов,
# остальные столбцы имеют много пропусков и не совсем репрезентативны

#### Столбцы, которых нет в трейне 2021

'super_gen' - аккумулирует и дублирует много информации с разных стобцов

'model_info' - дублирует полезную информацию из столбца car_url

'Владение' - длительность последнего или общего владения (не удалось спарсить, так как эти данные подгружаются AJAX)

'vendor' - продавец, пока уберем этот столбец

'equipment_dict' - аналог 'equipmentGroups'

'complectation_dict' - частично содержится в 'equipmentGroups'

In [189]:
train['equipment_dict'] = train['equipmentGroups']
train = train.drop(['equipmentGroups'], axis=1)

train.columns

#### Столбцы, которых нет в трейне 2020

'sell_id' - создадим его, наполнив 0

'car_url' - создадим, изменив model

'priceCurrency', 'parsing_unixtime', 'complectation_dict', 'equipment_dict', 'image', 'super_gen' - можно удалить

In [190]:
train_2020['sell_id'] = 0
train['sell_id'] = 0
train_2020['car_url'] = train_2020['model']

In [191]:
# список общих переменных

columns2021 = [x for x in s2021 if not x in no2021]
print('Общие для 2021')
print('Общее количество', len(columns2021))
print(columns2021)

columns2020 = [x for x in s2020 if not x in no2020]
print('Общие для 2020')
print('Общее количество', len(columns2020))
print(columns2020)

#### Удалим столбцы:

Из 2021

- не несут информации для цены - 'image','parsing_unixtime',
- слишком сложные в обработке - 'description', 'equipmentGroups'
- дублируют другие признаки -'model_name',
- единственное значение - 'priceCurrency', 'Состояние'

In [192]:
print('train - priceCurrency - ', train.priceCurrency.value_counts(), sep='\n')
print('test - priceCurrency - ', test.priceCurrency.value_counts(), sep='\n')
print()
print('train - Состояние - ', train['Состояние'].value_counts(), sep='\n')
print('test - Состояние - ', test['Состояние'].value_counts(), sep='\n')

In [193]:
to_del = ['image','parsing_unixtime','priceCurrency','description','model_name', 'Состояние','equipmentGroups']

In [194]:
columns = [x for x in columns2021 if not x in to_del] 
columns = ([x for x in columns if x in columns2020+['car_url', 'sell_id']] )

In [195]:
# У тестового датасета нет цены, добавим нулевую цену
test['price']=0

In [196]:
df_train_2020 = train_2020[columns]
df_train = train[columns]
df_test = test[columns]

#### Объединение train и test

In [197]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет

df_train_2020['sample'] = 1 # помечаем где у нас трейн
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).append(df_train_2020, sort=False).reset_index(drop=True)
data[data['sample']==0].shape

In [198]:
data.info()

In [199]:
# Переименуем признаки:

data.rename(columns={'productionDate': 'production_date',
                     'vehicleConfiguration': 'vehicle_configuration',
                     'vehicleTransmission': 'vehicle_transmission',
                     'name':'gear',
                     'Владельцы': 'owners_qty',
                     'Владение': 'ownership_time',
                     'ПТС': 'licence',
                     'Привод': 'type_of_drive',
                     'Руль': 'steering_wheel',
                     'Таможня': 'customs', 
                     'Price': 'price',
                     'bodyType': 'body_type', 
                     'engineDisplacement': 'engine_volume',
                     'enginePower': 'engine_power',
                     'fuelType': 'fuel_type',
                     'modelDate': 'model_date',
                     'numberOfDoors': 'number_of_doors',}, inplace=True)
data.columns

- body_type - тип кузова,
- brand - марка автомобиля,
- car_url - url страницы объявления
- color - цвет автомобиля,
- engine_displacement - объём двигателя,
- engine_volume - мощность двигателя,
- fuel_type - тип топлива,
- mileage - пробег,
- model_date - дата релиза модели,
- name - имя, введенное пользователем
- number_of_doors - количество дверей,
- production_date - дата производства автомобиля
- vehicle_configuration - конфигурация транспортного средства (ТС),
- vehicle_transmoission - тип коробки передач,
- owners_qty - количество владельцев,
- licence - паспорт ТС,
- type_of_drive - тип привода,
- steering_wheel - сторона руля,
- customs - этап растаможки,
- price - цена автомобиля, целевой параметр,
- sample - индикатор принадлежности данных к тесту (0) и трейну (1),

In [200]:
# Удалим дубликаты из датафрейма

data = data.drop_duplicates()
data[data['sample'] ==0].info()

### EDA

In [201]:
# числовые признаки
num_cols = []

# бинарные признаки
bin_cols = []

# категориальные признаки
cat_cols = []

# обработанные категориальные признаки
ready_cat_cols = []

In [202]:
data1 = data.copy() # сохраним данные в data1, чтобы зафиксировать изменения
data[data['sample']==0].shape

### CAR_URL

In [203]:
data.car_url

In [204]:
# Изменим столбец car_url на модель авто

value = data.car_url.apply(lambda x: x.lower() if len(x.split('/'))==1 else x.split('/')[7].lower())
idx = data.columns.get_loc('brand') 
data.insert(loc=idx+1, column='model_of_car', value=value )
data = data.drop('car_url',1)
data[data['sample'] ==0].info()

In [205]:
data['model_of_car']

In [206]:
cat_cols.append('model_of_car')

### bodyType

In [207]:
data['body_type'].unique()

In [208]:
# Переделаем названия
data['body_type'] = data['body_type'].astype(str).apply(lambda x: None if x.strip()=='' else x)
data['body_type'] = data['body_type'].apply(lambda x: x.split(' ')[0].lower())

In [209]:
data['body_type'].value_counts()

Самый популярный кузов внедорожник, на втором месте седан

посмотрим на пропуски

In [210]:
data['body_type'].isna().sum()

In [211]:
cat_cols.append('body_type')

### Brand

In [212]:
data['brand'].unique()

In [213]:
data['brand'].value_counts()

Наиболее популярные бренды: БМВ, Mercedes, Volkswagen

In [214]:
data['brand'].isnull().sum()

In [215]:
cat_cols.append('brand')

### color

In [216]:
data['color'].unique()

In [217]:
data['color'].value_counts()

Самый популярные цвета среди автомобилей - чёрный и белый

In [218]:
# переименуем признак:

color_dict = {'040001':'black', 
              'FAFBFB':'white',
              'CACECB':'silver',
              '97948F':'grey',
              'чёрный':'black',
              '0000CC':'blue',
              'белый':'white',
              '200204':'brown',
              'EE1D19':'red',
              'серебристый':'silver',
              'серый':'grey',
              'синий':'blue',
              '007F00':'green',
              'C49648':'beige',
              'красный':'red', 
              'коричневый':'brown',
              '22A0F8':'light_blue',
              'зелёный':'green',
              '660099':'purple',
              'DEA522':'gold',
              '4A2197':'violet',
              'бежевый':'beige',
              'FFD600':'yellow',
              'голубой':'light_blue',
              'FF8649':'orange',
              'золотистый':'gold',
              'пурпурный':'purple',
              'фиолетовый':'violet',
              'жёлтый':'yellow',
              'оранжевый':'orange',
              'FFC0CB':'pink',
              'розовый':'pink'
             }

data['color'] = data['color'].map(color_dict)

Разделим цвета по популярности. Самые популярные: 1, менее популярные: 2,еще менее популярные: 3, не популярные: 4

In [219]:
pop_color_dict = {
    'white': 1,
    'black': 1,
    'blue': 2,
    'silver': 2,
    'grey': 2,
    'brown': 3,
    'red': 3, 
    'green': 3,
    'beige': 3,
    'light_blue': 3,
    'purple': 4,
    'yellow': 4, 
    'orange': 4,
    'violet': 4, 
    'gold': 4, 
    'pink': 4
}
data['color'] = data['color'].map(pop_color_dict)

In [220]:
ready_cat_cols.append('color')

In [221]:
data['color'].isnull().sum()

In [222]:
data['color'].fillna(1, inplace=True)

### fuelType

In [223]:
data['fuel_type'].unique()

In [224]:
data['fuel_type'].value_counts()

Наиболее популярные машины на бензине, наименее - на газу

In [225]:
# дадим латинские названия

d = {
    'бензин': 'petrol', 
    'дизель': 'diesel', 
    'гибрид': 'hybrid', 
    'электро': 'electro', 
    'газ': 'gas'
}
data['fuel_type'] = data['fuel_type'].map(d)

In [226]:
cat_cols.append('fuel_type')

### model_date,production_date

In [227]:
np.sort(data['model_date'].unique())

In [228]:
data['model_date'].value_counts()

In [229]:
data['model_date'][data['model_date']< 1980].value_counts(dropna=False).sort_index(ascending=False)

Автомобилей, произведенных ранее 1980 года очень мало.

Примем эти автомобили за выбросы

In [230]:
# примем  автомобили старше 1980 г. за выбросы
data = data[~((data.model_date < 1980)&(data.sample==1))]

In [231]:
data['model_date'].isnull().sum()

In [232]:
data = data.dropna(subset=['model_date'])

In [233]:
data['production_date']

In [234]:
data['production_date'].isnull().sum()

In [235]:
# Введем новые признаки возраст модели и возраст машины

from datetime import date

today = date.today()
d1 = int(today.strftime("%Y"))

data['model_date'] = data['model_date'].astype('int')
idx = data.columns.get_loc('model_date') 
data.insert(loc=idx+1, column='model_d', value=d1 - data['model_date'] )
data.insert(loc=idx+1, column='production_d', value=d1 - data['production_date'] )

In [236]:
num_cols.append('model_date')
num_cols.append('production_date')
num_cols.append('model_d')
num_cols.append('production_d')

### Gear

In [237]:
data['gear'].unique()

In [238]:
data['gear'] = data['gear'].astype(str)
idx = data.columns.get_loc('color') 
data.insert(loc=idx+1, column='transmis', value= data['gear'].str.extract('([A][T]|[M][T]|[A][M][T]|[C][V][T])',
                                                expand=False).str.strip())

In [239]:
data['transmis'].value_counts()

Наиболее популряной коробкой передач является AT

In [240]:
data['transmis'].isnull().sum()

In [241]:
data['transmis']=data['transmis'].fillna('AT')

In [242]:
cat_cols.append('transmis')

In [243]:
data = data.drop('gear', 1)

### number_of_doors

In [244]:
data['number_of_doors'].unique()

In [245]:
mode = round(data['number_of_doors'].mean(),0)
mode

In [246]:
data['number_of_doors']=data['number_of_doors'].fillna(mode)

In [247]:
data['number_of_doors'].value_counts()

Наибольшее количество автомобилей имеет 5 дверей

In [248]:
num_cols.append('number_of_doors')

### vehicle_configuration

In [249]:
data['vehicle_configuration'].value_counts()

Получим данные о типе коробки передач

In [250]:
data['vehicle_configuration'] = data['vehicle_configuration'].astype(
                str).apply(lambda x: x if len(x) == 1 else x.split())

In [251]:
data['vehicle_configuration'] = data['vehicle_configuration'].apply(
    lambda x: x[0].lower() if len(x) == 1 else x[1].lower())

In [252]:
data['vehicle_configuration'].value_counts()

Больше всего машин с автоматической коробкой передач

In [253]:
cat_cols.append('vehicle_configuration')

### engine_volume

In [254]:
data['engine_volume'].value_counts(dropna=False).head(20)

In [255]:
data['engine_volume'] = data['engine_volume'].astype(str)

Получим объем двигателя

In [256]:
data['engine_volume'] = data['engine_volume'].str.extract('(\d.\d)',expand=False).str.strip()

data['engine_volume']=data['engine_volume'].astype(float)

In [257]:
data['engine_volume'] = data['engine_volume'].apply(lambda x: round(x/100,1) if (x>10) else round(x,1))

In [258]:
mean = round(data['engine_volume'].mean(),1)
mean

In [259]:
data[data['sample']==0]['engine_volume'].isnull().sum()

In [260]:
data['engine_volume'].fillna(mean, inplace=True)

In [261]:
data['engine_volume'].value_counts()

In [262]:
num_cols.append('engine_volume')

### engine_power

In [263]:
data['engine_power']

In [264]:
data['engine_power'] = data['engine_power'].astype(str).apply(lambda x: x.split()[0])
data['engine_power'] = data['engine_power'].astype(float)

In [265]:
data['engine_power'].value_counts()

In [266]:
data['engine_power'].value_counts().sort_index()

In [267]:
# Разобьём признак на категории:

def engine_power(x):
    if x < 100: x = 1
    elif 99 < x < 150: x = 2
    elif 149 < x < 200: x = 3
    elif 199 < x < 250: x = 4
    elif 249 < x < 300: x = 5
    elif 299 < x < 350: x = 6
    elif 349 < x < 400: x = 7
    elif 399 < x < 450: x = 8
    elif 449 < x < 500: x = 9
    elif 499 < x < 550: x = 10
    elif 549 < x < 600: x = 11
    else: x = 12
    return x  

In [268]:
data['engine_power'] = data['engine_power'].map(engine_power)

In [269]:
data['engine_power'].isnull().sum()

In [270]:
num_cols.append('engine_power')

### mileage

In [271]:
data['mileage']

In [272]:
data['mileage'] = data['mileage'].astype(str).apply(lambda x: x.replace('\xa0',''))
data['mileage'] = data['mileage'].astype(str).apply(lambda x: x.replace('км',''))
data['mileage'] = data['mileage'].apply(lambda x: int(x.replace(' ','')))
data['mileage']

In [273]:
data['mileage'].isna().sum()

In [274]:
num_cols.append('mileage')

### mileage_per_year

In [275]:
import math

value= round(data['mileage'].astype(int)/(data['model_d'].apply(lambda x: x if x>0 else x+1)),-2)
idx = data.columns.get_loc('licence') 
data.insert(loc=idx+1, column='mileage_per_year', value= value)
data['mileage_per_year'] = data['mileage_per_year'].apply(lambda x: x if x != math.inf else 0 )

In [276]:
data['mileage_per_year']

In [277]:
num_cols.append('mileage_per_year')

### type_of_drive

In [278]:
data['type_of_drive'].unique()

In [279]:
data['type_of_drive'].value_counts(dropna=False)

In [280]:
# дадим латинские названия
td = {
    'передний': 'front', 
    'полный': 'full', 
    'задний': 'back'
}
data['type_of_drive'] = data['type_of_drive'].map(td)

In [281]:
data['type_of_drive']=data['type_of_drive'].fillna('front')

Наибольшее ко-во машин имеет передний привод

In [282]:
cat_cols.append('type_of_drive')

### steering_wheel

In [283]:
data['steering_wheel'].value_counts(dropna=False)

In [284]:
steering_wheel_dict = {'LEFT':'left', 
                       'Левый':'left',
                       'RIGHT':'right', 
                       'Правый':'right',
                        }
data['steering_wheel'] = data['steering_wheel'].map(steering_wheel_dict)

In [285]:
data['steering_wheel'].value_counts()

In [286]:
data['steering_wheel'].isnull().sum()

In [287]:
bin_cols.append('steering_wheel')

### owners_qty

In [288]:
data['owners_qty'].value_counts(dropna=False)

In [289]:
data['owners_qty'] = data['owners_qty'].astype(
                str).apply(lambda x: x if len(x) == 1 else x.split())
data['owners_qty']

In [290]:
data['owners_qty']= data['owners_qty'].apply(
    lambda x: x[0].lower() if len(x) == 1 else x[0].lower())

In [291]:
data['owners_qty']=data['owners_qty'].astype(float)

In [292]:
data['owners_qty'].isnull().sum()

In [293]:
data['owners_qty'].fillna(2, inplace=True)

In [294]:
ready_cat_cols.append('owners_qty')

### licence

In [295]:
data['licence'].unique()

In [296]:
licence_dict={'Оригинал': 1,
             'Дубликат': 0,
             'ORIGINAL':1,
             'DUPLICATE':0}
data['licence'] = data['licence'].map(licence_dict)

In [297]:
data['licence'].value_counts(dropna=False)

Если лицензия не указана, то не будем ставить, что она есть, возможно машина без документов. Установим значение 2 - отличается от остальных

In [298]:
data['licence'] = data['licence'].fillna(2)

In [299]:
ready_cat_cols.append('licence')

### customs

In [300]:
data['customs'].value_counts()

всего один нерастаможенный автомобиль. Нет смысла держать этот признак в датасете

In [301]:
data = data.drop('customs', 1)

### vehicle_transmission

In [302]:
data['vehicle_transmission'].value_counts()

In [303]:
data = data.drop('vehicle_transmission',1)

Такой признак уже был

### Price

Уже делали обработку при учете инфляции для train 2021

In [304]:
data['price']

In [305]:
data['price'][(data['sample']==1)&(data['price'].isna())]

Так как это целевая переменная, удалим пустые значения

In [306]:
data.dropna(subset=['price'], inplace=True)

In [307]:
# Посмотрим, есть ли  дубликаты:

print('Количество дубликатов:',len(data) - len(data.drop_duplicates()))

In [308]:
data = data.drop_duplicates()
data.isna().sum()

## Анализ признаков

In [309]:
print(bin_cols)
print(num_cols)
print(cat_cols)
print(ready_cat_cols)

In [310]:
data.info()

### Анализ числовых переменных

In [311]:
# посмотрим на корреляцию признаков между собой
plt.figure(figsize=(10, 10))
sns.heatmap(data[num_cols + ['price']].corr().abs(), vmin=0, vmax=1,
            annot=True, fmt=".2f", cmap="YlGnBu")

Признаки model_date, production_date, model_d, production_d полностью коррелируют друг другом, поэтому оставим только один из них: production_d

In [312]:
data = data.drop(['model_date', 'production_date', 'model_d'], 1)

In [313]:
num_cols.remove('model_date')
num_cols.remove('production_date')
num_cols.remove('model_d')

In [314]:
for col in num_cols:
    visualizing_number(data, col)

In [315]:
# #определяем значимость наших переменных:

imp_cat = Series(mutual_info_classif(data[data['price'] >0][num_cols], 
                                     data[data['price'] >0]['price'],
                                     discrete_features = True), index = num_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')
plt.title('Значимость чиловых переменных для price')

### Анализ категориальных и бинарных переменных

In [316]:
data[ready_cat_cols + cat_cols + bin_cols]

vehicle_configuration и transmis означают одно и то же. Удалим vehicle_configuration

In [317]:
data = data.drop(['vehicle_configuration'], 1)

In [318]:
cat_cols.remove('vehicle_configuration')

In [319]:
for col in cat_cols:
    print(col)
    print(data[col].value_counts())
    print()

Разделим cat_cols:

- Если категорий мало, то будем обрабатывать их с помощью Dumming,
- Если категорий много, то - LabelEncoder

К категориям на LabelEncoder добавим bin_cols - бинарные признаки

In [320]:
data[bin_cols]

In [321]:
cat_oh_cols = ['fuel_type', 'transmis', 'type_of_drive']
cat_le_cols = ['model_of_car', 'body_type', 'brand', 'steering_wheel']

In [322]:
# Преобразуем все значения категориальных признаков в числа:

data = pd.get_dummies(data, columns=cat_oh_cols, dummy_na=False)

In [323]:
label_encoder = LabelEncoder()
for i in cat_le_cols:
    data[i] = label_encoder.fit_transform(data[i])
data.info()    

### Оценим влияние категориальных признаков на целевую переменную

In [324]:
cat_new_cols = data.columns
cat_new_cols = [x for x in cat_new_cols if x not in (num_cols + ['sell_id','price','sample'])]
data[cat_new_cols].info()

In [325]:
imp_num = imp_cat.copy()

In [326]:
imp_cat = Series(mutual_info_classif(data[data['price'].isna() == False][cat_new_cols], 
                                     data[data['price'].isna() == False]['price'],
                                     discrete_features = True), index = cat_new_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')
plt.title('Влияние категориальных признаков на price')

Объединим веса в один сериес. Нужно перевести его в list, но чтобы значения признаков стояли в том же порядке, что и в data

In [327]:
imp_weights = pd.concat([imp_num, imp_cat]).drop_duplicates()
cols = data.columns
weights = [imp_weights[col] for col in cols if col not in ['sell_id', 'price', 'sample']]
weights

### Разделение переменных

In [328]:
X = data.query('sample == 1').drop(['sample', 'price', 'sell_id'], axis=1)
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)
y = data.query('sample == 1')['price'].values
X_sub

In [329]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)
X_train

In [330]:
stsc = StandardScaler()
X_train = stsc.fit_transform(X_train)
X_test = stsc.transform(X_test)

# Добавим веса
X_train = X_train * weights
X_test = X_test * weights

Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели

In [331]:
np.log(y_train)

## LinearRegression

In [332]:
# lin_reg = LinearRegression().fit(X_train, y_train)
# y_pred = lin_reg.predict(X_test)
# print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

# Точность модели по метрике MAPE: 116.25%

## GradientBoosting

In [333]:
#gb = GradientBoostingRegressor(min_samples_split=2,
#                               learning_rate=0.03,
#                                max_depth=30,
#                               n_estimators=1000)
# gb.fit(X_train,y_train)

# Сначала попробуем без логорифмирования таргета

# y_pred = gb.predict(X_test)
# print(f"Точность модели по метрике MAPE_без лого: {(mape(y_test, y_pred))*100:0.2f}%")

# Точность модели по метрике MAPE_без лого: 20.26%

## Forest

In [334]:
# rf = RandomForestRegressor(n_estimators=1000,
#                            n_jobs=-1,
#                            max_depth=30,
#                            max_features='log2',
#                            random_state=RANDOM_SEED,
#                            oob_score=True)
# rf.fit(X_train,y_train)

In [335]:
# Сначала попробуем без логорифмирования таргета

#y_pred = rf.predict(X_test)
#print(f"Точность модели по метрике MAPE_без лого: {(mape(y_test, y_pred))*100:0.2f}%")

# Точность модели по метрике MAPE_без лого: 18.14%

## Xgboosting

In [336]:
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")

# Точность модели по метрике MAPE: 14.17% max_depth=8
# Точность модели по метрике MAPE: 13.82% max_depth=12
# Точность модели по метрике MAPE: 13.90% max_depth=18

# Submission

In [337]:
X_sub_st = stsc.transform(X_sub.drop('sell_id',1))
X_sub_st = X_sub_st * weights

pred = np.exp(xb.predict(X_sub_st))

In [338]:
predict_submission = np.round(pred,-3).astype('int')

In [339]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v16.csv', index=False)
sample_submission.head(10)

Вывод:
Лучший результат на валидации показала модель Xgboosting: 13.82%